In [1]:
import nltk
import praw
import pandas as pd
import datetime
import json

In [2]:
# Load credfile and display when last updated
credfile = 'credfile.json'
credfile_prefix = ''

# Read credentials to a dictionary
with open(credfile) as fh:
    creds = json.loads(fh.read())

print(f"[{datetime.datetime.now()}]" + f"{credfile} {'.' * 10} is being used as credfile")

[2019-04-06 16:12:07.957885]credfile.json .......... is being used as credfile


In [3]:
reddit = praw.Reddit(client_id=creds['client_id'],
                     client_secret=creds['client_secret'],
                     user_agent=creds['user_agent']
                    )

In [4]:
print(reddit.read_only)  # Output: True

True


## Take all top level comments and analyze

In [5]:
submission = reddit.submission(id='ba7uqx')

In [41]:
# alternatively, save comments as a list
top_level_comments = list(submission.comments)
all_comments = submission.comments.list()

In [42]:
print("Number of top level comments: ", len(top_level_comments))
print("Total number of comments:     ", len(all_comments))

Number of top level comments:  161
Total number of comments:      478


In [43]:
for comment in top_level_comments[:5]:
    print("Votes:  ", comment.score)
    print("Author: ", comment.author)
    print("Body:   ",  comment.body)
    print("===================")

Votes:   1
Author:  AutoModerator
Body:    **Mirrors / Alternate angles**

*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/soccer) if you have any questions or concerns.*
Votes:   520
Author:  Sinnedd
Body:    Damn, Costa must have insulted this guy’s entire family 
Votes:   228
Author:  FlyingArab
Body:    This was the most Diego Costa sequence ever
Votes:   276
Author:  Juggernautspammer
Body:    What the fuck could he have said to get a straight red holy shit 
Votes:   271
Author:  steven_vd
Body:    I am so confused


In [64]:
# iterate over top comments in the submission and\= create list of sentences
submission.comments.replace_more(limit=None)
top_level_comment_list = []
top_level_comment_string = ''
for top_level_comment in submission.comments[1:]: # Skip AutoMod comment
    top_level_comment_list.append(top_level_comment.body)
    top_level_comment_string += (str(top_level_comment.body)+'. ')

In [65]:
top_level_comment_list[0:5]

['Damn, Costa must have insulted this guy’s entire family ',
 'This was the most Diego Costa sequence ever',
 'What the fuck could he have said to get a straight red holy shit ',
 'I am so confused',
 'Simeone is gonna shank Diego at half time.']

In [66]:
top_level_comment_string[0:500]

"Damn, Costa must have insulted this guy’s entire family . This was the most Diego Costa sequence ever. What the fuck could he have said to get a straight red holy shit . I am so confused. Simeone is gonna shank Diego at half time.. Well, he didn't come back for very long.. Damn, the way atletico players surrounded the ref was inviting another red. Thats our boy. The way the referee gets crowded in la Liga disgusts me every time. . classic Diego Costa. lol wtf did he say to the ref. Gently whispe"

#### Polarity & Subjectivity using TextBlob

In [63]:
from textblob import TextBlob

analysis = TextBlob(top_level_comment_string)
print('Polarity score:     ', analysis.sentiment[0])
print('Subjectivity score: ', analysis.subjectivity)

Polarity score:      0.00040267719991222837
Subjectivity score:  0.4952159681076733


#### Readability score

In [72]:
from readability import Readability

r = Readability(top_level_comment_string)
fk = r.flesch_kincaid()

print("Flesch-kincaid score:       ", fk.score)
print("Flesch-kincaid grade level: ", fk.grade_level)

Flesch-kincaid score:        3.786948356807514
Flesch-kincaid grade level:  4


In [71]:
fk.grade_level

'4'

## Analyze tree of a single comment

In [30]:
top_level_comments[2].replies.list()

[Comment(id='ek9ly37'),
 Comment(id='ek9oyi9'),
 Comment(id='ek9mbrg'),
 Comment(id='ek9n6ob'),
 Comment(id='ek9mva9'),
 <MoreComments count=1, children=['ek9ne9x']>,
 Comment(id='ek9n5cw'),
 Comment(id='ek9n2gf'),
 Comment(id='ek9no6b'),
 Comment(id='ek9oo1w'),
 Comment(id='ek9nofh'),
 Comment(id='ek9nxd5'),
 <MoreComments count=3, children=['ek9nmuu']>,
 Comment(id='ek9o260'),
 Comment(id='ek9ocfo')]